In [1]:
if __name__ == "__main__":
    if hasattr(__builtins__,'__IPYTHON__') and ('google.colab' in str(get_ipython())):
        from google.colab import drive
        drive.mount('/content/drive')
        %cd /content/drive/MyDrive/PressureReliefWorkArea/SummerWork/
        !ls
    else: print('name main not colab')

name main not colab


In [29]:
# -n is not working apparently??
# %run -n DataProcessing.ipynb

__main__
name main not colab
__main__


In [ ]:
class SKModel:
    def __init__(self, sequence_length = None):
        # self.loader = JFLoader(file_path, sequence_length)
        self.sequence_length = sequence_length

    def skm_load_data(self, file_path):
        self.loader = JFLoader(file_path, self.sequence_length)

    def skm_train(self):
        pass
    
    def skm_evaluate(self):
        pass